#### Primary resources
- LangChain: https://www.langchain.com/
- LangChain chat memory: https://python.langchain.com/docs/how_to/chatbots_memory/


#### Additional resources
- How to use Google colab: https://colab.research.google.com/drive/16pBJQePbqkz3QFV54L4NIkOn1kwpuRrj

### Enter your Google API key

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

### Install required packages

In [ ]:
%pip install --upgrade --quiet langchain langgraph langchain-google-genai

### LLM config

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Add memory

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, RemoveMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

workflow = StateGraph(state_schema=MessagesState)

In [ ]:
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )

    system_message = SystemMessage(content=system_prompt)
    message_history = state["messages"][:-1]

    if len(message_history) >= 4:
        last_human_message = state["messages"][-1]
        
        # Invoke the model to generate conversation summary
        summary_prompt = (
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = model.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        # Delete messages that we no longer want to show up
        delete_messages = [RemoveMessage(id=m.id) for m in state["messages"]]
        
        # Re-add user message
        human_message = HumanMessage(content=last_human_message.content)
        
        # Call the model with summary & response
        response = model.invoke([system_message, summary_message, human_message])
        message_updates = [summary_message, human_message, response] + delete_messages
    else:
        message_updates = model.invoke([system_message] + state["messages"])

    return {"messages": message_updates}

In [ ]:
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### Example for chat history

In [ ]:
chat_history_example = [
    HumanMessage(content="Hey there! I'm Ricards."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

### Ask question

In [ ]:
def query_llm(question):
    ai_msg = app.invoke(
        {
            "messages": chat_history_example
            + [HumanMessage(question)]
        },
        config={"configurable": {"thread_id": "4"}},
    )
    print(ai_msg["messages"][-1])

In [ ]:
question = input("Please enter your question: ")
print(f"Question: {question}")

query_llm(question)